In [30]:
# Функция для подсчета YTM, принимающая аргументом словарь
def YTM_counter(dict):
  C = dict['C = coupon rate']
  N = dict['N = face value or nominal']
  PP = dict['PP = purchase price']
  T = dict['T = years to maturity\xa0\xa0']
  return round((C * N + (N - PP) / T) / ((N + PP) / 2), 4)


In [31]:
# Функция поиска nominal, при котором YTM станет равным нулю
def right_nominal(dict):
  C = dict['C = coupon rate']
  N = dict['N = face value or nominal']
  PP = dict['PP = purchase price']
  T = dict['T = years to maturity\xa0\xa0']
  return PP / (C * T + 1)

In [32]:
# Функция поиска бумаг с отрицательной доходностью к погашению - явная ошибка в дата сете
def errors(dict):
  errors = []
  for column in dict:
    if dict[column]["YTM"] < 0:
      errors.append(column)
  return errors

In [33]:
# Функция добавления в словарь исправленной строки.
def fixing(dict, need_to_fix):
  # Создаем типовой словарик, чтобы для каждой ошибки в дата сете инициализировать им новый ключ искомого словаря.
  dict_fix = {'C = coupon rate': 0, 'N = face value or nominal': 0, 'PP = purchase price': 0,
              'T = years to maturity\xa0\xa0': 0, 'YTM': 0}
  for instrument in need_to_fix:
    dict[instrument + ' fixed'] = dict_fix
    # Копируем в новый ключ значения старого, кроме nominal.
    dict[instrument + ' fixed']['C = coupon rate'] = dict[instrument]['C = coupon rate']
    dict[instrument + ' fixed']['PP = purchase price'] = dict[instrument]['PP = purchase price']
    dict[instrument + ' fixed']['T = years to maturity\xa0\xa0'] = dict[instrument]['T = years to maturity\xa0\xa0']
    # Nominal становится таким, что YTM = 0%.
    dict[instrument + ' fixed']['N = face value or nominal'] = int(right_nominal(dict[instrument]))
  return dict

In [34]:
# Импортируем библиотеку pandas для работы с таблицами.
import pandas as pd

df = pd.read_excel('data_set.xlsx', index_col = 0) # Считываем xlsx файл.
rows = df.to_dict("dict") # Переводим таблицу в словарь

# Добавляем строку с названием YTM и для каждой бумаги просчитываем данный индекс.
for column in df.columns:
    rows[column]["YTM"] = YTM_counter(rows[column])

# Находим бумаги с отрицательной доходностью к погашению и добавляем в словарь их исправленные версии.
need_to_fix = errors(rows)
rows = fixing(rows, need_to_fix)

# Переходим обратно от словаря к таблице, транспонируем для удобства и сортируем бумаги по YTM в убывающем порядке.
df = df.from_dict(rows)
df = df.sort_values(by = 'YTM', axis = 1, ascending = False)
df = df.transpose()

# Форматируем файл. Найденные бумаги с ошибками закрашиваем в красный цвет, а исправленные - в зеленый.
df = df.style.apply(lambda x: ['background: red' if x.name in need_to_fix else '' for i in x], axis=1)\
             .apply(lambda x: ['background: green' if 'fixed' in x.name else '' for i in x], axis=1)\
             .format({'C = coupon rate': '{:.2%}', 'N = face value or nominal': int, 'PP = purchase price': int,
                      'T = years to maturity\xa0\xa0': int, 'YTM': '{:.2%}'})
# Все готово!
df

,C = coupon rate,N = face value or nominal,PP = purchase price,T = years to maturity,YTM
Instrument 16,10.00%,14411,11347,9,13.83%
Instrument 10,9.00%,13637,11591,5,12.97%
Instrument 9,9.00%,14470,11853,7,12.74%
Instrument 17,10.00%,14036,12020,9,12.49%
Instrument 19,9.00%,11620,11185,10,9.55%
Instrument 7,9.00%,14004,13519,10,9.51%
Instrument 15,7.00%,13852,13055,5,8.39%
Instrument 14,9.00%,12345,12987,9,8.21%
Instrument 13,8.00%,11721,11714,8,8.01%
Instrument 2,6.00%,13183,11667,9,7.72%
